In [11]:
import cv2
import mediapipe as mp
import numpy as np
from pynput.mouse import Button,Controller as MouseController
from pynput.keyboard import Key,Controller as KeyboardController
import pyautogui
import time
mouse=MouseController()
keyboard=KeyboardController()

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
def zoom_in():
    # Simulate pressing the 'Ctrl' key and the '+' key to zoom in (browser example)
    pyautogui.hotkey('ctrl', '+')
    time.sleep(0.5)

def zoom_out():
    # Simulate pressing the 'Ctrl' key and the '-' key to zoom out (browser example)
    with keyboard.pressed(Key.ctrl_l):
        keyboard.press('-')
        keyboard.release('-')
        time.sleep(0.5)

def detect_touching(landmarks):
    finger_landmarks = landmarks.landmark

    # Define landmarks for thumb, index, middle, ring, and pinky fingers
    thumb_tip_id = 4
    index_tip_id = 8
    middle_tip_id = 12
    ring_tip_id = 16
    pinky_tip_id = 20

    if len(finger_landmarks) >= 21:  # Check if enough landmarks are detected
        thumb_tip = np.array([finger_landmarks[thumb_tip_id].x, finger_landmarks[thumb_tip_id].y])
        index_tip = np.array([finger_landmarks[index_tip_id].x, finger_landmarks[index_tip_id].y])
        middle_tip = np.array([finger_landmarks[middle_tip_id].x, finger_landmarks[middle_tip_id].y])
        ring_tip = np.array([finger_landmarks[ring_tip_id].x, finger_landmarks[ring_tip_id].y])
        pinky_tip = np.array([finger_landmarks[pinky_tip_id].x, finger_landmarks[pinky_tip_id].y])

        # Define a distance threshold for considering fingers as touching
        distance_threshold = 0.05  # Adjust this threshold as needed

        # Calculate the Euclidean distance between thumb and fingers
        index_distance = np.linalg.norm(thumb_tip - index_tip)
        middle_distance = np.linalg.norm(thumb_tip - middle_tip)
        ring_distance = np.linalg.norm(thumb_tip - ring_tip)
        pinky_distance = np.linalg.norm(thumb_tip - pinky_tip)

        # Check if any finger is touching
        if index_distance < distance_threshold:
            return 'index'
        elif middle_distance < distance_threshold:
            return 'middle'
        elif ring_distance < distance_threshold:
            return 'ring'
        elif pinky_distance < distance_threshold:
            return 'pinky'

    return None
# Set the frame size to 1280x720 (720p)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame and detect hands
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Detect if any of the fingers touch the thumb
                touching_finger = detect_touching(landmarks)

                if touching_finger == 'index':
                    cv2.putText(frame, 'Index Touching', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif touching_finger == 'middle':
                    cv2.putText(frame, 'Middle Touching', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif touching_finger == 'ring':
                    cv2.putText(frame, 'Ring Touching', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    zoom_in()
                elif touching_finger == 'pinky':
                    cv2.putText(frame, 'Pinky Touching', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    zoom_out()

        cv2.imshow('Hand Tracking', frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
            break

cap.release()
cv2.destroyAllWindows()


